In [9]:
pip install openpyxl==3.1.2

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load file
output_file = 'output_formatted_reviews_gpt4o_3_v6_1.csv'
answer_key_file = 'ABA Dataset - Label Topic (For Tasks 1 & 2).xlsx'

df_output = pd.read_csv(output_file)
df_answer_key = pd.read_excel(answer_key_file)

In [11]:
# df_answer_key.head(30)

In [12]:
print(df_answer_key.columns.tolist())

['Unnamed: 0', 'Title', 'PositiveReview', 'NegativeReview', 'Topic', 'Selected Content', 'Pos/Neg', 'Head', 'Body 1', 'Body 2', 'Body 3', 'Body 4', 'Body 5', 'Body 6', 'Body 7', 'Body 8', 'Body 9', 'Body 10', 'Cont. Body 1', 'Cont. Body 2', 'Cont. Body 3', 'Cont. Body 4', 'Cont. Body 5', 'Cont. Body 6', 'Cont. Body 7', 'Cont. Body 8', 'Cont. Body 9', 'Cont. Body 10', 'Concat', 'Unnamed: 29', 'Unnamed: 30']


In [13]:
# Clean answer key
# Rename these columns to match the wanted format:
df_answer_cleaned = df_answer_key[['Unnamed: 0', 'Topic', 'Selected Content', 'Pos/Neg']].rename(columns={
    'Unnamed: 0': 'ID',
    'Topic': 'Topics',
    'Selected Content': 'Text',
    'Pos/Neg': 'NegPos'
})

# df_answer_cleaned.head(30)

In [14]:
df_answer_cleaned[df_answer_cleaned['ID'] ==22]

,ID,Topics,Text,NegPos
46,22,Room,"Room was perfect, looked new, clean shiny tile...",Positive
47,22,Room,although might have needed another set of draw...,Negative
48,22,Food,"Ordered pizza one night and was not great,",Negative
49,22,Food,The restaurant area is nice and the staff very...,Positive


In [15]:
# Clean Output
# Define a function to clean the 'Topics' column in the model output:
# - Removes any trailing numbers from the topic string (e.g., 'Room1' -> 'Room')
def clean_topic(topic):
    return re.sub(r'\d+$', '', str(topic))

# Apply the clean_topic function to the 'Topics' column in df_output
# Store the result in a new column called 'Cleaned_Topics'
df_output['Cleaned_Topics'] = df_output['Topics'].apply(clean_topic)

# Create a cleaned version of the model output DataFrame by selecting relevant columns:
df_output_cleaned = df_output[['ID', 'Cleaned_Topics', 'Text', 'NegPos']].copy()
df_output_cleaned.rename(columns={'Cleaned_Topics': 'Topics'}, inplace=True)

# Remove rows where both 'Text' and 'NegPos' are NaN (missing output for this instance)
df_output_cleaned = df_output_cleaned[~(df_output_cleaned['Text'].isna() & df_output_cleaned['NegPos'].isna())]

# Optional to display first 30 row for checking
# df_output_cleaned.head(30)

# Aggregate the cleaned output to handle cases where multiple texts are generated for the same (ID, Topic, NegPos):
# - Group by 'ID', 'Topics', and 'NegPos'
# - For each group, concatenate all non-empty 'Text' values into a single string separated by commas
df_output_merged = df_output_cleaned.groupby(['ID', 'Topics', 'NegPos']).agg({
    'Text': lambda x: ', '.join(x.dropna().astype(str).str.strip())
}).reset_index()


# Optional: 
# View the first 30 rows of the merged output for inspection
# df_output_merged.head(30)

# Display the merged output for ID = 22 to check specific results
df_output_merged[df_output_merged['ID'] == 22]


,ID,Topics,NegPos,Text
59,22,Facility,Positive,The restaurant area is nice.
60,22,Food,Negative,"Ordered pizza one night and was not great, too..."
61,22,Food,Positive,I'm sure other food is better.
62,22,Room,Negative,might have needed another set of drawers if th...
63,22,Room,Positive,"Room was perfect, looked new, clean shiny tile..."
64,22,Staff,Positive,the staff very accommodating.


In [16]:
# df_output_merged .tail(30)

In [17]:
# Merge the cleaned answer key (df_answer_cleaned) and the cleaned model output (df_output_merged)

# - Before merging, rename columns to explicitly indicate source (answer or output):
#     - In the answer key: 'Text' -> 'Text_answer', 'NegPos' -> 'NegPos_answer'
#     - In the model output: 'Text' -> 'Text_output', 'NegPos' -> 'NegPos_output'
# - Merge on 'ID', 'Topics', and sentiment labels ('NegPos_answer' vs. 'NegPos_output')
# - Use an outer join to include all cases from both the answer key and the model output,
#   even when there is no match (this helps identify both missing predictions and extra outputs)
df_compare = pd.merge(
    df_answer_cleaned.rename(columns={'NegPos': 'NegPos_answer', 'Text': 'Text_answer'}),
    df_output_merged.rename(columns={'NegPos': 'NegPos_output', 'Text': 'Text_output'}),
    left_on=['ID', 'Topics', 'NegPos_answer'],
    right_on=['ID', 'Topics', 'NegPos_output'],
    how='outer'
)

# df_compare.head(30)
df_compare[df_compare['ID'] == 8]

,ID,Topics,Text_answer,NegPos_answer,NegPos_output,Text_output
17,8,Facility,NaN,NaN,Positive,renovated
18,8,Location,nice location,Positive,Positive,nice location
19,8,Staff,lovely personal,Positive,Positive,lovely personal


In [18]:
# Topic match function
# Define a function to check if the topic extraction is correct (topic match check):
def topic_match(row):
    text_answer = row['Text_answer']
    text_output = row['Text_output']

    # Case: both are NaN or empty → drop
    if (pd.isna(text_answer) or str(text_answer).strip() == '') and \
       (pd.isna(text_output) or str(text_output).strip() == ''):
        return 'drop'
    
    # Case: both are present → True
    elif (not pd.isna(text_answer) and str(text_answer).strip() != '') and \
         (not pd.isna(text_output) and str(text_output).strip() != ''):
        return True
    
    # Case: one is missing → False
    else:
        return False

# Apply match check and create a new column called 'Topic_Match' that contains 'drop', True, or False for each pair
df_compare['Topic_Match'] = df_compare.apply(topic_match, axis=1)

# Filter out rows where 'drop'
df_compare_filtered = df_compare[df_compare['Topic_Match'] != 'drop'].copy()

# Save
comparison_output_file = 'Comparison_Result_Topic_Match_3_shot_v6_1.csv'
# comparison_output_file = 'test.csv'
df_compare_filtered.to_csv(comparison_output_file, index=False)

print(f"Filtered comparison result saved as '{comparison_output_file}'")
df_compare.head(20)




,ID,Topics,Text_answer,NegPos_answer,NegPos_output,Text_output,Topic_Match
0,1,Food,"Tasty food on the first floor, comfortable res...",Positive,Positive,"Tasty food on the first floor, comfortable res...",True
1,1,Location,"close to the airport, to very clean beach.",Positive,Positive,"close to the airport, to very clean beach.",True
2,1,Room,NaN,NaN,Positive,"New, comfortable apartments.",False
3,1,Staff,Staff is extremely helpful and easy to communi...,Positive,Positive,Staff is extremely helpful and easy to communi...,True
4,2,Facility,NaN,NaN,Positive,"I like hotels with a family atmosphere, cozy a...",False
5,2,Food,"Also, the breakfast was amazing, fresh and han...",Positive,Positive,"the breakfast was amazing, fresh and handmade!...",True
6,2,Room,NaN,NaN,Negative,although not big.,False
7,2,Room,"The room was very clean, well decorated and mo...",Positive,Positive,"The room was very clean, well decorated and mo...",True
8,2,Staff,The staff was very nice and helpful!,Positive,Positive,The staff was very nice and helpful! all staff...,True
9,3,Location,the location is great and near the airport. bu...,Positive,Positive,the location is great and near the airport. bu...,True


In [19]:
df_compare[df_compare['ID'] == 22]

,ID,Topics,Text_answer,NegPos_answer,NegPos_output,Text_output,Topic_Match
60,22,Facility,NaN,NaN,Positive,The restaurant area is nice.,False
61,22,Food,"Ordered pizza one night and was not great,",Negative,Negative,"Ordered pizza one night and was not great, too...",True
62,22,Food,The restaurant area is nice and the staff very...,Positive,Positive,I'm sure other food is better.,True
63,22,Room,although might have needed another set of draw...,Negative,Negative,might have needed another set of drawers if th...,True
64,22,Room,"Room was perfect, looked new, clean shiny tile...",Positive,Positive,"Room was perfect, looked new, clean shiny tile...",True
65,22,Staff,NaN,NaN,Positive,the staff very accommodating.,False


In [20]:
num_answer_topics = len(df_answer_cleaned)
num_output_topics = len(df_output_merged)

print(f"Total topics in Answer Key: {num_answer_topics}")
print(f"Total topics in Model Output: {num_output_topics}")


Total topics in Answer Key: 381
Total topics in Model Output: 583


In [21]:
total = len(df_compare)
correct = (df_compare['Topic_Match'] == True).sum()
accuracy = correct / total if total > 0 else 0

print(f"Total comparisons: {total}")
print(f"Correct matches: {correct}")
print(f"Topic Classification Accuracy: {accuracy:.2%}")

Total comparisons: 610
Correct matches: 354
Topic Classification Accuracy: 58.03%


In [22]:
# df_filtered = df_compare[df_compare['Topic_Match'] != 'drop']

# # Calculate True Positives (TP):
# # - Both human and model extracted text for the same (ID, Topic, NegPos) pair
# TP = (df_filtered['Topic_Match'] == True).sum()

# FP = ((df_filtered['Topic_Match'] == False) & (df_filtered['Text_output'].notna()) & (df_filtered['Text_answer'].isna())).sum()
# FN = ((df_filtered['Topic_Match'] == False) & (df_filtered['Text_output'].isna()) & (df_filtered['Text_answer'].notna())).sum()

# precision = TP / (TP + FP) if (TP + FP) > 0 else 0
# recall = TP / (TP + FN) if (TP + FN) > 0 else 0
# f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

# print(f"Precision: {precision:.2%}")
# print(f"Recall: {recall:.2%}")
# print(f"F1 Score: {f1:.2%}")
